# Prequisite

*Prequisite Week 3 - Power of Data Scraping*

1. Download WebDriver (Edge/Chrome)  
https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/?form=MA13LH  
https://developer.chrome.com/docs/chromedriver/downloads  
sesuaikan dengan versi browser masing-masing yang terinstall (bisa di cek di pengaturan browser)  
  
2. Basic Concept of HTML and CSS  
Karena kita nanti casenya web scraping, kita perlu tahu juga apa itu HTML dan CSS, salah satu refferensinya bisa cek link berikut  
https://www.hostinger.com/tutorials/what-is-html  
https://blog.lewagon.com/skills/what-is-css/  

# Import Library

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Web Driver Configuration

In [38]:
# Pilih sesuai web browser
# options = webdriver.EdgeOptions()
options = webdriver.ChromeOptions() 
options.use_chromium = True
# options.add_argument("headless")  # Untuk menjalankan browser dalam mode headless
driver = webdriver.Edge(options=options)
wait = WebDriverWait(driver, 5)

# Determine Target Location

Akses Maps Menggunakan [Place Id](https://chromewebstore.google.com/detail/maps-place-id-search-tool/gdnnaahojechcmemagbbbbnoiieolafp) (Search Place Id Finder buat cari id dari lokasi)

<div style="display: flex;">
    <img src="https://i.ibb.co.com/ncgY5Kv/111.jpg" style="max-width: 50%;">
    <img src="https://i.ibb.co.com/vJX3mWP/222.jpg" style="max-width: 50%;">
</div>

In [39]:
tus_place_id = 'ChIJHVqSy9H71y0RnwWbLgvLvh0'
base_url = 'https://www.google.com/maps/place/?q=place_id:'
driver.get(f'{base_url}{tus_place_id}')

# Open Review Tab

In [40]:
# Get List of Tab Menu
list_menu = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Gpq6kf.fontTitleSmall')))

# filter elemen berdasarkan teksnya dan berhenti pada yang pertama ditemukan
text_target = ['Ulasan', 'Rating'] # karena kan ada menu overview dan about (jadi kita pilih rating)
target_element = None
for el in list_menu:
    if el.text in text_target:
        target_element = el
        print('Tab Review Found')
        break

Tab Review Found


In [41]:
target_element.click()

# Scrool Sidebar

In [42]:
# Find Sidebar
sidebar = driver.find_element(By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde")
print(sidebar)

<selenium.webdriver.remote.webelement.WebElement (session="c410f29ed08767f38ed8742e46347136", element="f.97D2604FF72865EA0C7F4D4858B06B9E.d.B2A56F8BF8241A0BB670F6DBE70539D8.e.80")>


In [43]:
# Fungsi untuk menggulir sidebar ke bawah
def scroll_sidebar_to_bottom(time_sleep):
    # Ambil tinggi sidebar
    last_height = driver.execute_script("return arguments[0].scrollHeight;", sidebar)
    
    while True:
        # Gulir sidebar ke bawah
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", sidebar)
        
        # Tunggu beberapa detik untuk memuat konten baru
        time.sleep(time_sleep)
        # Value Time Sleep Sesuaikan dengan koneksi dan device, bisa jadi bisa lebih cepat atau lambat untuk scrooling dan melakukan rendering content)
        
        # Hitung tinggi sidebar baru setelah menggulir
        new_height = driver.execute_script("return arguments[0].scrollHeight;", sidebar)
        
        # Jika tinggi sidebar baru sama dengan tinggi sebelumnya, berarti sudah mencapai bagian bawah
        if new_height == last_height:
            break
        
        last_height = new_height

# Panggil fungsi untuk menggulir sidebar sampai ke bawah
scroll_sidebar_to_bottom(2)

# Find the Element You Want to Scrape

<div style="text-align: center;">
    <img src="https://yourimageshare.com/ib/L9Su4s43Zv.webp" width="450">
</div>

`To scrape review URL, obtain the geo-coordinates manually in Google Maps by right-clicking near the red pin on the map`

>Don't worry, Google Map coordinates are not that strict. The important thing is to ensure that your coordinates are still within the scope of the location you're referring to

<div style="text-align: center;">
    <img src="https://i.ibb.co.com/1LvmxHG/Whats-App-Image-2024-05-17-at-22-22-29-c8e48943.jpg" width="400">
</div>

In [59]:
print(len(driver.find_elements(By.CSS_SELECTOR, 'div.d4r55')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'div.RfnDt')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'span.kvMYJc[aria-label*="bintang"]')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'span.rsqaWe')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'span.wiI7pd')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'span.pkWtMe')))
print(len(driver.find_elements(By.CSS_SELECTOR, 'button.al6Kxe')))

287
258
287
287
106
75
287


In [67]:
for i in range(0, 5):
    print(driver.find_elements(By.CSS_SELECTOR, 'button.al6Kxe')[i].get_attribute('data-href'))

https://www.google.com/maps/contrib/110809995417901392155/reviews?hl=id
https://www.google.com/maps/contrib/103472688785147558501/reviews?hl=id
https://www.google.com/maps/contrib/103110328363506692527/reviews?hl=id
https://www.google.com/maps/contrib/116378066798139972794/reviews?hl=id
https://www.google.com/maps/contrib/101202826206442055427/reviews?hl=id


In [44]:
def get_digits(num, digit):
    return tuple(round(n, digit) for n in num)

`Adjust the geo-coordinate to your case` 

In [45]:
geo_coordinate = (get_digits((-7.3110645918643185, 112.72901710435903), 6)) 
review_id_test = driver.find_element(By.CLASS_NAME, 'jftiEf').get_attribute('data-review-id')
base_review_url = f'https://www.google.com/maps/reviews/@{geo_coordinate[0]},{geo_coordinate[1]},17z/data=!3m1!4b1!4m6!14m5!1m4!2m3!1s{review_id_test}!2m1!1s0x0:0x4ceb8bb356c12235?hl=id&entry=ttu'
print(base_review_url)

https://www.google.com/maps/reviews/@-7.311065,112.729017,17z/data=!3m1!4b1!4m6!14m5!1m4!2m3!1sChZDSUhNMG9nS0VJQ0FnSUREOUlUYU1BEAE!2m1!1s0x0:0x4ceb8bb356c12235?hl=id&entry=ttu


# Get Card Comments > Looping > Scrape All 

In [ ]:
def extract_status(string):
    parts = string.split("·")
    # ['Local Guide', '378 ulasan','1.954 foto']
    
    guide_type = 'Non Local Guide'
    review_count = None
    photo_count = None

    for part in parts:
        clean_part = part.strip().split()
        # ['1.954', 'ulasan']
        if any(word in clean_part for word in ['review', 'ulasan']):
            review_count = int(clean_part[0].replace('.',''))
        elif any(word in clean_part for word in ['photo', 'foto']):
            photo_count = int(clean_part[0].replace('.',''))
        else:
            # Jika tidak mengandung kata 'ulasan' atau 'foto', itu mungkin menjadi tipe guide
            guide_type = ' '.join(clean_part)

    # print(guide_type, "\t", review_count, "\t", photo_count)
    return guide_type, review_count, photo_count


In [49]:
print(extract_status('Local Guide · 378 ulasan · 1.954 foto'))
print(extract_status('378 ulasan · 1.954 foto'))
print(extract_status('1.954 foto'))

('Local Guide', 378, 1954)
('Non Local Guide', 378, 1954)
('Non Local Guide', None, 1954)


In [69]:
names = []
status = [] # To be parsed by type of guide (local/non-local) | number of reviews | number of photos
guide_types = []
guide_review_count = []
guide_photo_count = []
rating = []
rating_dates = []
descriptions = []
likes = []
profile_links = []
review_links = []


comment_cards = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf')
for i, cc in enumerate(comment_cards):
    # print(i)
    names.append(cc.find_element(By.CSS_SELECTOR, 'div.d4r55').text)
    
    try:
        st = cc.find_element(By.CSS_SELECTOR, 'div.RfnDt').text
        g_type, g_review_count, g_photo_count = extract_status(st)
        guide_types.append(g_type)
        guide_review_count.append(g_review_count)
        guide_photo_count.append(g_photo_count)
    except:
        err_val = '-'
        guide_types.append(err_val)
        guide_review_count.append(err_val)
        guide_photo_count.append(err_val)
        
    rating.append(cc.find_element(By.CSS_SELECTOR, 'span.kvMYJc[aria-label*="bintang"]').get_attribute("aria-label").split()[0])
    rating_dates.append(cc.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text)
    
    try:
        dsc = cc.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
        descriptions.append(dsc)
    except:
        descriptions.append("-")
    
    try:
        like = cc.find_element(By.CSS_SELECTOR, 'span.pkWtMe').text
        likes.append(like)
    except:
        likes.append(0)
        
    profile_links.append(cc.find_element(By.CSS_SELECTOR, 'button.al6Kxe').get_attribute("data-href"))
    try:
        review_id = cc.get_attribute('data-review-id')
        base_review_url = f'https://www.google.com/maps/reviews/@{geo_coordinate[0]},{geo_coordinate[1]},17z/data=!3m1!4b1!4m6!14m5!1m4!2m3!1s{review_id}!2m1!1s0x0:0x4ceb8bb356c12235?hl=id&entry=ttu'
        review_links.append(base_review_url)
    except:
        print('REVIEW ID ERROR')
    

`use try and catch to handle unavailable element`

In [70]:
print(len(names))
print(len(guide_types))
print(len(guide_review_count))
print(len(guide_photo_count))
print(len(rating))
print(len(rating_dates))
print(len(descriptions))
print(len(likes))
print(len(profile_links))
print(len(review_links))

287
287
287
287
287
287
287
287
287
287


In [72]:
datas = [names, guide_types, guide_review_count, guide_photo_count, rating, rating_dates, descriptions, likes, profile_links, review_links]
for d in datas:
    print(d[-1])

Yohanes Deo
-
-
-
5
5 tahun lalu
-
0
https://www.google.com/maps/contrib/111176234443502779454/reviews?hl=id
https://www.google.com/maps/reviews/@-7.311065,112.729017,17z/data=!3m1!4b1!4m6!14m5!1m4!2m3!1sChdDSUhNMG9nS0VJQ0FnSURnZ09qS3V3RRAB!2m1!1s0x0:0x4ceb8bb356c12235?hl=id&entry=ttu


# BINGO :> :> :> :> :>

# Export 

In [73]:
import pandas as pd

In [74]:
data = {
    'name': names,
    'guide_type': guide_types,
    'guide_review_count': guide_review_count,
    'guide_photo_count': guide_photo_count,
    'rating': rating,
    'rating_date': rating_dates,
    'description': descriptions,
    'likes': likes,
    'profile_link': profile_links,
    'review_link': review_links
}

df = pd.DataFrame(data)
df.head()

,name,guide_type,guide_review_count,guide_photo_count,rating,rating_date,description,likes,profile_link,review_link
0,A.R. Bagas Danang Haditio,Local Guide,79,767,1,sebulan lalu,Kangen suasana shalt ied di lapangan Telkom td...,1,https://www.google.com/maps/contrib/1108099954...,https://www.google.com/maps/reviews/@-7.311065...
1,Adi Prasetyo,Local Guide,378,1954,5,10 bulan lalu,Salah satu kampus it telkom di daerah strategi...,6,https://www.google.com/maps/contrib/1034726887...,https://www.google.com/maps/reviews/@-7.311065...
2,zulvia Rachmawati,Non Local Guide,4,None,5,1 hari lalu,"Ka, kalo lulusan di Tel U surabaya gimana ya?B...",0,https://www.google.com/maps/contrib/1031103283...,https://www.google.com/maps/reviews/@-7.311065...
3,Rosyid Abdillah,Local Guide,9,34,5,setahun lalu,salah satu opsi terbaik untuk studi lanjut dib...,5,https://www.google.com/maps/contrib/1163780667...,https://www.google.com/maps/reviews/@-7.311065...
4,Arda Erico Yuda,Local Guide,17,1,5,4 bulan lalu,"Pengalaman belajar disini sangat menyenangkan,...",4,https://www.google.com/maps/contrib/1012028262...,https://www.google.com/maps/reviews/@-7.311065...


In [75]:
df.to_csv('gugel_mep.csv', index=False)

# Note: Pay attention to the language you use (to select and filtering data)

# How to Easy Scrapping Website?
# `Understand the Structure of Website (Inspect Element is the Key :>)`

### Build by Whisnumurty Galih Ananta

### Find me  
- [LinkedIn](https://www.linkedin.com/in/whisnumurtyga/)
- [GitHub](https://github.com/whisnumurtyga)